# 앙상블 학습과 랜덤포레스트02

### 배깅과 페이스팅

다양한 분류기를 만드는 방법중 하나는 각기 다른 훈련 알고리즘을 사용하는 것이라고 언급했습니다. 

또 다른 방법으로 같은 알고리즘을 사용하지만 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습 시키는 방법이 있습니다.

훈련 세트에서 중복을 허용하여 샘플링 하는 방식을 __배깅(bagging, bootstrap agreegating의 줄임말)__ 이라 하며,  
중복을 허용하지 않고 샘플링 하는 방식을 __페이스팅(pasting)__ 이라고 한다.  
통계학에서는 중복을 허용한 리샘플링을 __부트스트래핑__ 이라고 한다.

다시 말해 배깅과 페이스팅에서는 같은 훈련 샘플을 여러 개의 예측기에 걸쳐 사용할 수 있다.  
하지만, 배깅만이 한 예측기를 위해 훈련 샘플을 여러 번 샘플링 할 수 있다.  


모든 예측기가 훈련을 마치면, 앙상블은 모든 예측기의 예측을 모아서 새로운 샘플에 대한 예측을 만든다.  
수집 함수는 전형적으로 분류일 떄는 통계적 최반값(statistical mode, 직접 투표 분류기 처럼 가장 많은 예측 결과) 이고, 회귀에 대해서는 평균을 계산한다.  

In [20]:
# 아래는 여러 분류기를 조합하여, 사이킷 런의 투표 기반 분류기(VotingClassifier)를 만들고 훈련시키는 코드이다.
# 훈련 세트는 moons 데이터 셋이다.

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

x, y = make_moons(n_samples = 500, noise = 0.30 , random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=42)

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# 배깅 classifer
from sklearn.ensemble import BaggingClassifier


log_clf = LogisticRegression(solver='liblinear', random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=10, random_state=42)
svm_clf = SVC(gamma='auto', random_state=42)



bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, max_samples=100, bootstrap=True, n_jobs=-1)
bag_clf.fit(x_train,y_train)
y_pred = bag_clf.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.92


In [27]:

# 의사결정나무 말고도 다른 Classifier에도 사용가능
bag_clf = BaggingClassifier(SVC(), n_estimators=500, max_samples=100, bootstrap=True, n_jobs=-1)
bag_clf.fit(x_train,y_train)
y_pred = bag_clf.predict(x_test)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test,y_pred))

0.856


BaggingClassifier 는 기반이 되는 분류기가 결정 트리 분류기처럼 클래스 확률을 추정할 수 있으면 (predict_proba()메소드가 있으면) 직접 투표 대신 자동으로 간접 투표 방식을 사용한다. 